# Optimierung und Evaluierung des Sequential bow Modells

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sn
import h5py
import pydot

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, confusion_matrix, classification_report, log_loss
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier

from wordcloud import WordCloud
from PIL import Image
from PIL import ImageFilter

from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform, normal, qlognormal, randint

from keras.models import Sequential
from keras.layers import Dense, Dropout, LeakyReLU, Activation
from keras.regularizers import l2, l1
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping
from keras.models import load_model
from keras.utils import np_utils, plot_model
from keras.optimizers import Adagrad
from keras.losses import binary_crossentropy

Using TensorFlow backend.


In [2]:
def data_bow():
    x_test = pd.read_hdf("../build/preprocessed/bow_data_500.hdf5",key="test")
    x_train = pd.read_hdf("../build/preprocessed/bow_data_500.hdf5",key="train")
    y_test = x_test.label
    y_train = x_train.label
    x_test= x_test.drop('label',axis=1)
    x_train = x_train.drop('label',axis=1)
    return x_train, y_train, x_test, y_test

## First Test

In [ ]:
x_test = pd.read_hdf("../build/preprocessed/bow_data_500.hdf5",key="test")
x_train = pd.read_hdf("../build/preprocessed/bow_data_500.hdf5",key="train")
y_test = x_test.label
y_train = x_train.label
x_test= x_test.drop('label',axis=1)
x_train = x_train.drop('label',axis=1)

In [ ]:
dim = x_train.shape[1]
model = Sequential()
model.add(Dense(1770, kernel_regularizer=l1(2.0036577552673407e-06), input_dim=dim))
model.add(Activation('relu'))
model.add(Dense(9,kernel_regularizer=l2(0.05407632514834404)))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', metrics=['accuracy'],

            optimizer='Adagrad')

model.summary()
    
result = model.fit(x_train.values, y_train.values,
                    batch_size=64,
                    epochs=100,
                    verbose=2,
                    validation_split=0.3)

In [ ]:
def plot_history(network_history):
    plt.figure()
    plt.xlabel('Epochen')
    plt.ylabel('Verlust')
    plt.plot(network_history.history['loss'])
    plt.plot(network_history.history['val_loss'])
    plt.legend(['Training', 'Validierung'])
    plt.show()
    plt.close()
plot_history(result)

## Analyse des Inputs

In [ ]:
x_test = pd.read_hdf("../build/preprocessed/bow_data_500.hdf5",key="test")
x_train = pd.read_hdf("../build/preprocessed/bow_data_500.hdf5",key="train")
X = x_test.append(x_train)

$$max\left(\frac{(\bar{w_1}-\bar{w_2})^2}{s_1^2+s_2^2}\right)$$

In [ ]:
mu_real = X[X.label==1].mean()
mu_fake = X[X.label==0].mean()
sorted_words = ((mu_real-mu_fake)**2/(np.var(X[X.label==1])**2 + np.var(X[X.label==0])**2)).sort_values(ascending=False).index

words_plot = pd.DataFrame({'Fake':X[X.label==0][sorted_words[1:11]].mean(),'Real':X[X.label==1][sorted_words[1:11]].mean()})
words_plot.plot(kind='bar')
plt.ylabel(r"$\overline{w}$")
plt.legend()
plt.tight_layout()
plt.savefig("../build/plots/data_visualisation.pdf")
plt.show()

In [ ]:
news = pd.read_csv('../data/mixed_news/news_dataset.csv')
news = news.dropna(subset=['title','content'])
news = news[news.content != ' ']
news = news[news.title != ' ']
text_len_real  =  [len(c) for c in news[news['label']=='real'].content]
text_len_fake = [len(c) for c in news[news['label']=='fake'].content]
print("Mittlere textlänge Real: ",np.mean(text_len_real))
print("Mittlere Textlänge Fake: ",np.mean(text_len_fake))
print("Real ist %d länger wie Fake: " % (np.mean(text_len_real)-np.mean(text_len_fake)))

## How could the Hyperparameter be distributed

### Strukture

Größe der ersten und zweiten hidden layer

In [ ]:
x = np.round(np.random.lognormal(6,0.5,10000)/10)*10
plt.hist(x,bins=100)
plt.xlim(0,5000)
plt.show()

Größer der dritten hidden layer

In [ ]:
x = np.round(np.random.lognormal(4,0.5,10000)/1)*1
plt.hist(x,bins=100)
plt.show()

### Regularization

In [ ]:
x =np.random.uniform(0,0.1,10000)
plt.hist(x,bins=100)
plt.show()

## Model creation

In [ ]:
def model_structure(x_train, y_train, x_test, y_test):
    dim = x_train.shape[1]
    model = Sequential()
    model.add(Dense(int({{qlognormal(6,0.5,10)}}), input_dim=dim))
    model.add(Activation('relu'))
    
    if {{choice(['three', 'four'])}} == 'four':
        model.add(Dense(int({{qlognormal(6,0.5,10)}})))
        model.add(Activation('relu'))
        
    model.add(Dense(int({{qlognormal(4,0.5,1)}})))
    model.add(Activation('relu'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    model.compile(loss='binary_crossentropy', metrics=['accuracy'],
                  optimizer='adam')

    result = model.fit(x_train.values, y_train.values,
              batch_size=64,
              epochs=30,
              verbose=2,
              validation_split=0.3)
    validation_acc = np.amax(result.history['val_acc']) 
    print('Best validation acc of epoch:', validation_acc)
    return {'loss': -validation_acc, 'status': STATUS_OK, 'model': model}

In [ ]:
def model_training(x_train, y_train, x_test, y_test):
    dim = x_train.shape[1]
    model = Sequential()
    model.add(Dense(1770, input_dim=dim))
    model.add(Activation('relu'))
    model.add(Dense(9))
    model.add(Activation('relu'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    model.compile(loss={{choice(['hinge','binary_crossentropy','squared_hinge'])}}, metrics=['accuracy'],
                  optimizer={{choice(['adam','AdaDelta','Adagrad'])}})

    result = model.fit(x_train.values, y_train.values,
              batch_size=64,
              epochs=30,
              verbose=2,
              validation_split=0.3)
    validation_acc = np.amax(result.history['val_acc']) 
    print('Best validation acc of epoch:', validation_acc)
    return {'loss': -validation_acc, 'status': STATUS_OK, 'model': model}

In [ ]:
def model_regularization(x_train, y_train, x_test, y_test):
    dim = x_train.shape[1]
    model = Sequential()
    model.add(Dense(1770, kernel_regularizer=l1({{uniform(0,0.1)}}), input_dim=dim))
    model.add(Activation('relu'))
    model.add(Dense(9,kernel_regularizer=l2({{uniform(0,0.1)}})))
    model.add(Activation('relu'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    model.compile(loss='binary_crossentropy', metrics=['accuracy'],
                  optimizer='Adagrad')

    result = model.fit(x_train.values, y_train.values,
                      batch_size=64,
                      epochs=30,
                      verbose=2,
                      validation_split=0.3)
    validation_acc = np.amax(result.history['val_acc']) 
    print('Best validation acc of epoch:', validation_acc)
    return {'loss': -validation_acc, 'status': STATUS_OK, 'model': model}

In [ ]:
def model_optimizer(x_train, y_train, x_test, y_test):
    dim = x_train.shape[1]
    model = Sequential()
    model.add(Dense(1770, kernel_regularizer=l1(2.0*10**(-6)), input_dim=dim))
    model.add(Activation('relu'))
    model.add(Dense(9,kernel_regularizer=l2(0.05407632514834404)))
    model.add(Activation('relu'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    model.compile(loss='binary_crossentropy', metrics=['accuracy'],
                  optimizer=Adagrad(lr={{uniform(0,1)}}, epsilon=None, decay={{uniform(0,1)}}))

    result = model.fit(x_train.values, y_train.values,
                      batch_size=64,
                      epochs=30,
                      verbose=2,
                      validation_split=0.3)
    validation_acc = np.amax(result.history['val_acc']) 
    print('Best validation acc of epoch:', validation_acc)
    return {'loss': -validation_acc, 'status': STATUS_OK, 'model': model}

## Optimization with hyperopt
Algorithm: Tree of Parzen Estimators
Optimierung in 3 Schritten:
    - Struktur (Tiefe (2 oder 3 hidden Layers) und Breite)
    - Training (loss function und optimizer)
    - Regularizierung ( L1 für die erste Layer und L2 für 2 und 3)

In [ ]:
trials = Trials()
best_run, best_model = optim.minimize(model=model_structure,
                                      data=data_bow,
                                      algo=tpe.suggest,
                                      max_evals=50,
                                      trials=trials,
                                     notebook_name='Sequential_bow')
print("Best performing model chosen hyper-parameters:")
print(best_run)
best_model.save('../model/best_Hyperopt_NN_bow_struct_500.hdf5')

In [ ]:
trials = Trials()
best_run, best_model = optim.minimize(model=model_training,
                                      data=data_bow,
                                      algo=tpe.suggest,
                                      max_evals=15,
                                      trials=trials,
                                     notebook_name='Sequential_bow')
print("Best performing model chosen hyper-parameters:")
print(best_run)
best_model.save('../model/best_Hyperopt_NN_bow_training_500.hdf5')

In [ ]:
trials = Trials()
best_run, best_model = optim.minimize(model=model_regularization,
                                      data=data_bow,
                                      algo=tpe.suggest,
                                      max_evals=80,
                                      trials=trials,
                                     notebook_name='Sequential_bow')
print("Best performing model chosen hyper-parameters:")
print(best_run)
best_model.save('../model/best_Hyperopt_NN_bow_regularization2_500.hdf5')

In [ ]:
trials = Trials()
best_run, best_model = optim.minimize(model=model_optimizer,
                                      data=data_bow,
                                      algo=tpe.suggest,
                                      max_evals=100,
                                      trials=trials,
                                     notebook_name='Sequential_bow')
print("Best performing model chosen hyper-parameters:")
print(best_run)
best_model.save('../model/best_Hyperopt_NN_bow_optimizer_500.hdf5')

Beste Regularization: 

L1 in der ersten Layer = 0.00010911483516010123

L2 in der zweiten Layer = 0.027248712155710758

lr = 0.12056175158012145

epsilon = 0.1999709211230266

Die Optimierung des Lernrate führt jedoch zu einem Optimizer, der leicht in Nebenmaxima stecken bleibt (hier das Nebenmaxima, dass alles auf real zu schätzen). Daher wird der Standard Lerner verwendet



## Evaluation of best model

### Train best model
Neues Training des besten Modells, welches Optimiert bezüglich der Hyperparameter ist

In [3]:
def plot_history(network_history):
    plt.figure()
    plt.xlabel('Epochen')
    plt.ylabel('Verlust')
    plt.plot(network_history.history['loss'])
    plt.plot(network_history.history['val_loss'])
    plt.legend(['Training', 'Validierung'])
    #plt.show()
    plt.savefig("../build/plots/bow/500/history_bow_best.pdf")
    plt.close()

In [4]:
X_train, Y_train, X_test, Y_test = data_bow()
best_model = load_model('../model/best_Hyperopt_NN_bow_regularization2_500.hdf5')
model = Sequential.from_config(best_model.get_config())
print(model.summary())
model.get_config()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_124 (Dense)            (None, 1770)              886770    
_________________________________________________________________
activation_124 (Activation)  (None, 1770)              0         
_________________________________________________________________
dense_125 (Dense)            (None, 9)                 15939     
_________________________________________________________________
activation_125 (Activation)  (None, 9)                 0         
_________________________________________________________________
dense_126 (Dense)            (None, 1)                 10        
_________________________________________________________________
activation_126 (Activation)  (None, 1)                 0         
Total params: 902,719


{'name': 'sequential_42',
 'layers': [{'class_name': 'Dense',
   'config': {'name': 'dense_124',
    'trainable': True,
    'batch_input_shape': (None, 500),
    'dtype': 'float32',
    'units': 1770,
    'activation': 'linear',
    'use_bias': True,
    'kernel_initializer': {'class_name': 'VarianceScaling',
     'config': {'scale': 1.0,
      'mode': 'fan_avg',
      'distribution': 'uniform',
      'seed': None}},
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'kernel_regularizer': {'class_name': 'L1L2',
     'config': {'l1': 0.00010911483695963398, 'l2': 0.0}},
    'bias_regularizer': None,
    'activity_regularizer': None,
    'kernel_constraint': None,
    'bias_constraint': None}},
  {'class_name': 'Activation',
   'config': {'name': 'activation_124',
    'trainable': True,
    'activation': 'relu'}},
  {'class_name': 'Dense',
   'config': {'name': 'dense_125',
    'trainable': True,
    'units': 9,
    'activation': 'linear',
    'use_bias': True,
    'kerne

In [5]:
filepath = '../model/best_Hyperopt_NN_bow_trained_500.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True)
#stopping = EarlyStopping(monitor='val_loss', min_delta=0, restore_best_weights=True)
model.compile(loss='binary_crossentropy',
              optimizer='Adagrad',
              metrics=['accuracy'])
history = model.fit(X_train.values, Y_train.values, validation_split=0.3,
                    epochs=100,batch_size=64, callbacks=[checkpoint])#, stopping])

Train on 13672 samples, validate on 5860 samples
Epoch 1/100
13672/13672 [==============================] - 10s 765us/step - loss: 1.2189 - acc: 0.5622 - val_loss: 0.8206 - val_acc: 0.5635

Epoch 00001: val_loss improved from inf to 0.82060, saving model to ../model/best_Hyperopt_NN_bow_trained_500.hdf5
Epoch 2/100
13672/13672 [==============================] - 10s 767us/step - loss: 0.8000 - acc: 0.5818 - val_loss: 0.7560 - val_acc: 0.6005

Epoch 00002: val_loss improved from 0.82060 to 0.75602, saving model to ../model/best_Hyperopt_NN_bow_trained_500.hdf5
Epoch 3/100
13672/13672 [==============================] - 9s 673us/step - loss: 0.7501 - acc: 0.6140 - val_loss: 0.7219 - val_acc: 0.6343

Epoch 00003: val_loss improved from 0.75602 to 0.72191, saving model to ../model/best_Hyperopt_NN_bow_trained_500.hdf5
Epoch 4/100
13672/13672 [==============================] - 10s 716us/step - loss: 0.7223 - acc: 0.6514 - val_loss: 0.6599 - val_acc: 0.7623

Epoch 00004: val_loss improved from

13672/13672 [==============================] - 7s 496us/step - loss: 0.3876 - acc: 0.8926 - val_loss: 0.3996 - val_acc: 0.8823

Epoch 00034: val_loss did not improve from 0.39860
Epoch 35/100
13672/13672 [==============================] - 8s 607us/step - loss: 0.3876 - acc: 0.8913 - val_loss: 0.3982 - val_acc: 0.8817

Epoch 00035: val_loss improved from 0.39860 to 0.39818, saving model to ../model/best_Hyperopt_NN_bow_trained_500.hdf5
Epoch 36/100
13672/13672 [==============================] - 7s 513us/step - loss: 0.3846 - acc: 0.8925 - val_loss: 0.3929 - val_acc: 0.8869

Epoch 00036: val_loss improved from 0.39818 to 0.39286, saving model to ../model/best_Hyperopt_NN_bow_trained_500.hdf5
Epoch 37/100
13672/13672 [==============================] - 8s 566us/step - loss: 0.3813 - acc: 0.8934 - val_loss: 0.3911 - val_acc: 0.8846

Epoch 00037: val_loss improved from 0.39286 to 0.39106, saving model to ../model/best_Hyperopt_NN_bow_trained_500.hdf5
Epoch 38/100
13672/13672 [===============

13672/13672 [==============================] - 11s 801us/step - loss: 0.3428 - acc: 0.8996 - val_loss: 0.3638 - val_acc: 0.8860

Epoch 00069: val_loss improved from 0.36391 to 0.36378, saving model to ../model/best_Hyperopt_NN_bow_trained_500.hdf5
Epoch 70/100
13672/13672 [==============================] - 10s 697us/step - loss: 0.3428 - acc: 0.9007 - val_loss: 0.3619 - val_acc: 0.8863

Epoch 00070: val_loss improved from 0.36378 to 0.36191, saving model to ../model/best_Hyperopt_NN_bow_trained_500.hdf5
Epoch 71/100
13672/13672 [==============================] - 10s 736us/step - loss: 0.3410 - acc: 0.8989 - val_loss: 0.3615 - val_acc: 0.8860

Epoch 00071: val_loss improved from 0.36191 to 0.36150, saving model to ../model/best_Hyperopt_NN_bow_trained_500.hdf5
Epoch 72/100
13672/13672 [==============================] - 11s 829us/step - loss: 0.3405 - acc: 0.8980 - val_loss: 0.3605 - val_acc: 0.8865

Epoch 00072: val_loss improved from 0.36150 to 0.36054, saving model to ../model/best_Hy

In [6]:
plot_history(history)

## Evaluation of best model
Betrachten des trainierten Modells. Darstellung der Confusion Matrix, Overtraining Plot und ROC Curve

In [7]:
best_model = load_model('../model/best_Hyperopt_NN_bow_trained_500.hdf5')
y_pred = best_model.predict(X_test.values, batch_size=64, verbose=1)
y_pred_train = best_model.predict(X_train.values, batch_size=64, verbose=1)
y_pred_bool = np.round(y_pred[:,0])
Y_test = pd.DataFrame({"label":Y_test,"prediction":y_pred[:,0],"prediction_bool":y_pred_bool})
Y_train = pd.DataFrame({"label":Y_train,"prediction":y_pred_train[:,0]})

19532/19532 [==============================] - 2s 98us/step


In [8]:
print(classification_report(Y_test['label'], Y_test['prediction_bool']))
print("Binary Cross Entropie: ",log_loss(Y_test.label, Y_test.prediction))

#Confusion Matrix
cnfn_matrix = pd.crosstab(Y_test['label'], Y_test['prediction_bool'], rownames=['Wahrheit'], colnames=['Schätzung'])
print(cnfn_matrix)
cnfn_matrix.columns = ['Fake','Real']
cnfn_matrix = cnfn_matrix.rename_axis("Schätzung", axis="columns")
cnfn_matrix.rename(index = {0.0: "Fake", 1.0:'Real'}, inplace = True) 
cnfn_matrix = cnfn_matrix/Y_test.shape[0]
sn.heatmap(cnfn_matrix, annot=True , cmap='viridis')
#plt.show()
plt.savefig("../build/plots/bow/500/cnfsn_mtx_bow_best_nn.pdf")
plt.close()

#Overtraining test
plt.hist(Y_test.prediction[Y_test.label == 0],label="fake test", alpha = 0.4, color = "r",density=True)
plt.hist(Y_train.prediction[Y_train.label == 0],label='fake train', alpha = 0.4, color = 'r', histtype='step',density=True)
plt.hist(Y_test.prediction[Y_test.label == 1],label = "real test",alpha = 0.4, color = "b",density=True)
plt.hist(Y_train.prediction[Y_train.label == 1],label='real train', alpha = 0.4, color = 'b', histtype='step',density=True)

plt.xlabel("Geschätzte Likelihood")
plt.ylabel("Dichte")
plt.legend(loc='upper center')
#plt.show()
plt.savefig("../build/plots/bow/500/prob_bow_best_nn.pdf")
plt.close()

fpr = dict()
tpr = dict()
roc_auc = dict()
fpr, tpr, _ = roc_curve(Y_test.label, Y_test.prediction)
roc_auc = auc(fpr, tpr)

plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC Kurve (AUC = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc="lower right")
#plt.show()
plt.savefig("../build/plots/bow/500/roc_Hyperopt_bow_best_nn.pdf")
plt.close()

              precision    recall  f1-score   support

           0       0.86      0.87      0.87      3658
           1       0.90      0.89      0.89      4713

   micro avg       0.88      0.88      0.88      8371
   macro avg       0.88      0.88      0.88      8371
weighted avg       0.88      0.88      0.88      8371

Binary Cross Entropie:  0.316260521772017
Schätzung   0.0   1.0
Wahrheit             
0          3184   474
1           515  4198


## Interpretation

### Wordcloud confusion matrix

Darstellung der Wordhäufigkeiten in WordClouds für FP,FN,TP,TN getrennt

In [9]:
FP = Y_test[(Y_test.prediction_bool== 1) & (Y_test.label == 0)]
FN = Y_test[(Y_test.prediction_bool== 0) & (Y_test.label == 1)]
TP = Y_test[(Y_test.prediction_bool== 1) & (Y_test.label == 1)]
TN = Y_test[(Y_test.prediction_bool== 0) & (Y_test.label == 0)]
X_FP = X_test.loc[FP.index]
X_FN = X_test.loc[FN.index]
X_TP = X_test.loc[TP.index]
X_TN = X_test.loc[TN.index]

In [10]:
def plotWordcloud_cnfn(TN,FN,FP,TP):    
    TN = TN.sum().to_dict()
    FN = FN.sum().to_dict()
    FP = FP.sum().to_dict()
    TP = TP.sum().to_dict()
    
    pad = 5
    
    fig = plt.figure(figsize=(15,10),dpi=100)

    ax = plt.subplot(2, 2, 1)
    wordcloud = WordCloud(background_color='black',
                          width=1920,
                          height=1080,
                          mask=np.array(Image.open('../data/pictures/trump_silhouette.png'))
                          ).generate_from_frequencies(TN)
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.margins(x=0, y=0)

    ax = plt.subplot(2, 2, 2)
    wordcloud = WordCloud(background_color='black',
                          width=1920,
                          height=1080,
                          mask= np.array(Image.open('../data/pictures/trump_silhouette.png'))
                          ).generate_from_frequencies(FP)
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.margins(x=0, y=0)

    ax = plt.subplot(2, 2, 3)
    wordcloud = WordCloud(background_color='black',
                          width=1920,
                          height=1080,
                          mask=np.array(Image.open('../data/pictures/USA.jpg'))
                          ).generate_from_frequencies(FN)
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.margins(x=0, y=0)

    plt.subplot(2, 2, 4)
    wordcloud = WordCloud(background_color='black',
                          width=1920,
                          height=1080,
                          mask=np.array(Image.open('../data/pictures/USA.jpg'))
                          ).generate_from_frequencies(TP)
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    
    plt.figtext(0.5, 1.09, r"Prediction", {'fontsize': 30},
         horizontalalignment='center',
         verticalalignment='top')
    plt.figtext(0.25, 1.02, r"fake", {'fontsize': 20},
         horizontalalignment='center',
         verticalalignment='bottom',)
    plt.figtext(0.75, 1.02, r"real", {'fontsize': 20},
         horizontalalignment='center',
         verticalalignment='bottom',)
    
    plt.figtext(-0.07, 0.5, r"Actual", {'fontsize': 30},
         horizontalalignment='left',
         verticalalignment='center',
         rotation=90)
    plt.figtext(0.00, 0.75, r"fake", {'fontsize': 20},
         horizontalalignment='right',
         verticalalignment='center',)
    plt.figtext(0.00, 0.25, r"real", {'fontsize': 20},
         horizontalalignment='right',
         verticalalignment='center',)
    
    plt.margins(x=0, y=0)
    plt.tight_layout()
    #plt.show()
    plt.savefig("../build/plots/bow/500/cnfn_wordcloud.pdf", bbox_inches = 'tight')
    plt.close()

In [11]:
plotWordcloud_cnfn(X_TN,X_FN,X_FP,X_TP)

### Wordcloud fake real news

In [ ]:
def plotWordcloud(content,t):
    if(t!=""):
       mask = np.array(Image.open('../data/pictures/'+t))
    else:
        mask=None
        

    content = content.sum().to_dict()
    wordcloud = WordCloud(background_color='black',
                      width=1920,
                      height=1080,
                      mask=mask
                      ).generate_from_frequencies(content)
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.margins(x=0, y=0)

In [ ]:
X = X_test.append(X_train)
y = Y_test.append(Y_train)
plt.figure(dpi=200)
plotWordcloud(X[y.label==0],"trump_silhouette.png")
#plt.show()
plt.savefig("../build/plots/fake_wordcloud.pdf",bbox_inches='tight',pad_inches = 0)
plt.close()
plt.figure(dpi=200)
plotWordcloud(X[y.label==1],"USA.jpg")
#plt.show()
plt.savefig("../build/plots/real_wordcloud.pdf",bbox_inches='tight',pad_inches = 0)
plt.close()

## Untersuchung der first layer

Summieren der Beträge aller Gewichte eines Neurons ohne Offset und anschließende Darstellung in WordCloud

In [12]:
words = X_test.columns
first_weights = best_model.layers[0].get_weights()[0]
first_weights = pd.DataFrame(first_weights.transpose())
first_weights.columns = words
first_weightabs = np.abs(first_weights)
first_weightsum = first_weightabs.sum(axis=0)
content = np.abs(first_weightsum).to_dict()
wordcloud = WordCloud(background_color='black',
                      width=1920,
                      height=1080
                      ).generate_from_frequencies(content)
plt.figure(dpi=100)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.margins(x=0, y=0)
#plt.show()
plt.savefig("../build/plots/bow/500/weights_wordcloud.pdf")
plt.close()

Untersuchung der Confusion Matrix mithilfe der wichtigsten 10 Wörter 

In [13]:
sorted_weights = first_weightsum.sort_values(ascending=False)
best_words = sorted_weights[:10].index

fig = plt.figure(figsize=(15,10),dpi=100)

ax = plt.subplot(2, 2, 1)
(X_TN[best_words].sum()/X_TN.shape[0]).plot(kind='bar',label="TN",color='r')
plt.margins(x=0, y=0)
plt.ylabel("mittlere Worthäufigkeit")
plt.ylim(0.01,60)
plt.yscale("log")

ax = plt.subplot(2, 2, 2)
(X_FP[best_words].sum()/X_FP.shape[0]).plot(kind='bar',label="FP",color='g')
plt.margins(x=0, y=0)
plt.ylabel("mittlere Worthäufigkeit")
plt.ylim(0.01,60)
plt.yscale("log")

ax = plt.subplot(2, 2, 3)
(X_FN[best_words].sum()/X_FN.shape[0]).plot(kind='bar',label="FN",color='k')
plt.margins(x=0, y=0)
plt.ylabel("mittlere Worthäufigkeit")
plt.ylim(0.01,60)
plt.yscale("log")

plt.subplot(2, 2, 4)
(X_TP[best_words].sum()/X_TP.shape[0]).plot(kind='bar',label="TP",color='b')
plt.margins(x=0, y=0)
plt.ylabel("mittlere Worthäufigkeit")
plt.ylim(0.01,60)
plt.yscale("log")
    
plt.figtext(0.5, 1.09, r"Schätzung", {'fontsize': 30},
     horizontalalignment='center',
     verticalalignment='top')
plt.figtext(0.25, 1.02, r"Fake", {'fontsize': 20},
     horizontalalignment='center',
     verticalalignment='bottom',)
plt.figtext(0.75, 1.02, r"Real", {'fontsize': 20},
     horizontalalignment='center',
     verticalalignment='bottom',)
    
plt.figtext(-0.07, 0.5, r"Wahrheit", {'fontsize': 30},
     horizontalalignment='left',
     verticalalignment='center',
     rotation=90)
plt.figtext(0.00, 0.75, r"Fake", {'fontsize': 20},
     horizontalalignment='right',
     verticalalignment='center',)
plt.figtext(0.00, 0.25, r"Real", {'fontsize': 20},
     horizontalalignment='right',
     verticalalignment='center',)
    
plt.margins(x=0, y=0)
plt.tight_layout()
#plt.show()
plt.savefig("../build/plots/bow/500/cnfn_hist.pdf", bbox_inches = 'tight')
plt.close()

### RF Vergleichsmodell
Training einer RF auf dem bow Input und Evaluierung

In [14]:
RF = RandomForestClassifier(n_estimators=100, max_depth=10,random_state=0,criterion='entropy')
RF.fit(X_train.values,Y_train.label.values)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [15]:
y_pred_bool_RF = RF.predict(X_test.values)
y_pred_RF = RF.predict_proba(X_test.values)
y_pred_RF = y_pred_RF[:,1]
y_pred_train_RF = RF.predict_proba(X_train.values)
y_pred_train_RF = y_pred_train_RF[:,1]
Y_test['prediction_RF'] = y_pred_RF
Y_test['prediction_bool_RF'] = y_pred_bool_RF
Y_train['prediction_RF'] = y_pred_train_RF

In [16]:
print(classification_report(Y_test.label, Y_test.prediction_bool_RF))

#Confusion Matrix
cnfn_matrix = pd.crosstab(Y_test.label, Y_test.prediction_bool_RF, rownames=['Wahrheit'], colnames=['Schätzung'])
print(cnfn_matrix)
cnfn_matrix.columns = ['Fake','Real']
cnfn_matrix = cnfn_matrix.rename_axis("Schätzung", axis="columns")
cnfn_matrix.rename(index = {0.0: "Fake", 1.0:'Real'}, inplace = True) 
cnfn_matrix = cnfn_matrix/Y_test.shape[0]
sn.heatmap(cnfn_matrix, annot=True , cmap='viridis')
#plt.show()
plt.savefig("../build/plots/bow/500/RF/cnfsn_mtx_bow_best_nn.pdf")
plt.close()

#Overtraining test
bin_edges = np.linspace(0,1,11)
plt.hist(Y_test.prediction_RF[Y_test.label == 0],label="fake test", alpha = 0.4, color = "r",density=True,bins=bin_edges)
plt.hist(Y_train.prediction_RF[Y_train.label == 0],label='fake train', alpha = 0.4, color = 'r', histtype='step',density=True,bins=bin_edges)
plt.hist(Y_test.prediction_RF[Y_test.label == 1],label = "real test",alpha = 0.4, color = "b",density=True,bins=bin_edges)
plt.hist(Y_train.prediction_RF[Y_train.label == 1],label='real train', alpha = 0.4, color = 'b', histtype='step',density=True,bins=bin_edges)

plt.xlabel("Geschätzte Likelihood")
plt.ylabel("Dichte")
plt.legend(loc='upper center')
#plt.show()
plt.savefig("../build/plots/bow/500/RF/prob_bow_best_nn.pdf")
plt.close()

fpr_RF = dict()
tpr_RF = dict()
roc_auc_RF = dict()
fpr_RF, tpr_RF, _ = roc_curve(Y_test.label, Y_test.prediction_RF)
roc_auc_RF = auc(fpr_RF, tpr_RF)

plt.figure()
lw = 2
plt.plot(fpr_RF, tpr_RF, color='darkorange',
         lw=lw, label='ROC Kurve (AUC = %0.2f)' % roc_auc_RF)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc="lower right")
#plt.show()
plt.savefig("../build/plots/bow/500/RF/roc_Hyperopt_bow_best_nn.pdf")
plt.close()

              precision    recall  f1-score   support

           0       0.87      0.73      0.79      3658
           1       0.81      0.91      0.86      4713

   micro avg       0.83      0.83      0.83      8371
   macro avg       0.84      0.82      0.83      8371
weighted avg       0.84      0.83      0.83      8371

Schätzung     0     1
Wahrheit             
0          2679   979
1           405  4308


### Comparison
Vergleich des Sequential mit dem RF in der ROC Curve

In [17]:
plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='Sequential (AUC = %0.2f)' % roc_auc)
plt.plot(fpr_RF, tpr_RF, color='darkred',
         lw=lw, label='RandomForest (AUC = %0.2f)' % roc_auc_RF)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc="lower right")
#plt.show()
plt.savefig("../build/plots/bow/500/roc_comparison.pdf")
plt.close()

## Draw Modell

In [ ]:
from graphviz import Digraph

m = Digraph(name='Modell', node_attr={'shape': 'record'})
m.attr(rankdir='LR')

m.node("input", r"𝐄𝐢𝐧𝐠𝐚𝐧𝐠𝐬 𝐋𝐚𝐠𝐞 | 500 Wörter ")
m.node("first", r"𝐄𝐫𝐬𝐭𝐞 𝐋𝐚𝐠𝐞|Neuronen: 1770|Aktivierung: ReLu|L1: 0.00011")
m.node("second", r"𝐙𝐰𝐞𝐢𝐭𝐞 𝐋𝐚𝐠𝐞|Neuronen: 9|Aktivierung: ReLu|L2: 0.0272")
m.node("output", r"𝐀𝐮𝐬𝐠𝐚𝐧𝐠𝐬 𝐋𝐚𝐠𝐞|Neuronen: 1 |Aktivierung: Sigmoid")

m.edge('input', 'first')
m.edge('first', 'second')
m.edge('second', 'output')

m.render('../build/plots/bow/500/modell_scheme', view=True) 